In [1]:
import sys
sys.version
sys.version_info

sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)

In [2]:
from keras.utils import to_categorical
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
TRAIN_LAB_PATH = "./data/train_labels.csv"
PREPROCESSED_TRAINING = "./data/processedData.npy"
PREPROCESSED_KAGGLE = "./data/processed_kaggle.npy"

In [ ]:
CATEGORIES = ['apple', 'empty', 'moustache', 'mouth', 'mug', 'nail', 'nose', 'octagon', 'paintbrush', 'panda', 'parrot', 'peanut', 'pear', 'pencil', 'penguin', 'pillow', 'pineapple', 'pool', 'rabbit', 'rhinoceros', 'rifle', 'rollerskates', 'sailboat', 'scorpion', 'screwdriver', 'shovel', 'sink', 'skateboard', 'skull', 'spoon', 'squiggle']

def getIndexOf(category):
    return CATEGORIES.index(category)

def getCategoryOf(index):
    return CATEGORIES[index]

def load(infile):
    unformatted_images = np.load(infile, encoding='bytes')
    formatted = []
    for i,img in enumerate(unformatted_images):
        formatted.append([i, img[0]])
    return formatted

def formatXData(X, xDimension = 40):
    X = np.asarray(X)
    # Convert to matrix form
    X = X.reshape(-1, xDimension, xDimension, 1)
    # Convert to float
    X = X.astype('float32')
    # Scale pixel values between 0 and 1
    X = X / 255
    return X.astype('float32')

def addRotations(X,y):
    newX = []
    newY = []
    for i,XMatrix in enumerate(X):
        newX.append(XMatrix)
        newY.append(y[i])
        newX.append(np.rot90(XMatrix, 1))
        newY.append(y[i])
        newX.append(np.rot90(XMatrix, 2))
        newY.append(y[i])
        newX.append(np.rot90(XMatrix, 3))
        newY.append(y[i])
    return np.asarray(newX),np.asarray(newY)

from scipy.ndimage import rotate, zoom

# Centers the image
def center(img):
    minCoord, maxCoord = boundingBox(img)
    xLength = maxCoord[0] - minCoord[0]
    yLength = maxCoord[1] - minCoord[1]
    newImg = [[0 for j in range(len(img[i]))] for i in range(len(img))]
    
    startX = int((len(img) - xLength) / 2)
    startY = int((len(img[0]) - yLength) / 2)
    for i,x in enumerate(range(startX, startX + xLength + 1)):
        for j,y in enumerate(range(startY, startY + yLength + 1)):
            newImg[x][y] = img[minCoord[0]+i][minCoord[1]+j]
    return newImg

# Crop out all of the white space. If you want square dimensions, then it will pad white space.
def cropWhite(img, isSquare = False, whiteBoundary = True):
    minCoord, maxCoord = boundingBox(img)
    xLength = maxCoord[0] - minCoord[0] + 3
    yLength = maxCoord[1] - minCoord[1] + 3
    if isSquare:
        xLength = max(xLength, yLength)
        yLength = max(xLength, yLength)
    
    newImg = [[0 for j in range(yLength + 1)] for i in range(xLength + 1)]
    for i in range(xLength):
        for j in range(yLength):
            # Check for case where we are out of bounds for cropped white + square
            if (minCoord[0] + i) >= len(img) or (minCoord[1] + j) >= len(img[i]):
                newImg[i + 1][j + 1] = 0
            else:
                newImg[i + 1][j + 1] = img[minCoord[0] + i][minCoord[1] + j]
    return newImg

# rescale image to square of height, width = dimension    
def rescale(img, dimension, order = 0):
    cropped = cropWhite(img)
    height = len(cropped)
    width = len(cropped[0])
    zoomFactor = dimension / max(height, width)
    return zoom(img, zoomFactor, order=order)

def getRotations(x, y, interval_deg=30):
    newX = []
    newY = []
    newX.append(x)
    newY.append(y)
    deg = interval_deg
    while deg < 360:
#         newX.append(np.rot90(XMatrix, 1))
        newX.append(rotate(deg))
        newY.append(y)
        deg += interval_deg
    return np.asarray(newX),np.asarray(newY)

def preprocessRotImgs(x, y, rescaleDimension = 40, order = 1, interval_deg=30):
    rotX, Y = getRotations(x, y)
    for i in range(0, len(rotX)):
        croppedImg = cropWhite(rotX[i], True)
        centered_img = center(croppedImg)
        rescaled_img = rescale(centered_img, rescaleDimension, order)
        rotX[i] = rescaled_img
    return rotX, Y

def getRotData(X, Y, rescaleDimension = 40, interval_deg=30):
    newX = []
    newY = []
    for i in range(0,len(X)):
        newx, newy = preprocessRotImgs(X[i], Y[i])
        newX.extend(newx)
        newY.extend(newy)
    return newX, newY

def formatData(images, labels, xDimension = 40):
    categories = list(set(labels['Category']))
    X = []
    y = []
    for i, img in enumerate(images):
        label = labels.at[i,'Category']
        categoryNum = getIndexOf(label)
        X.append(img[1])
        y.append(categoryNum)
    y = to_categorical(y)
    X = formatXData(X, xDimension)
    return X.astype('float32'), y

def split(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1) 
    return X_train, y_train, X_test, y_test, X_val, y_val


In [ ]:
training_imgs = load(PREPROCESSED_TRAINING)

In [ ]:
labels = pd.read_csv(TRAIN_LAB_PATH)

In [ ]:
training_imgs, labels = getRotData(training_imgs, labels)

In [ ]:
X,y = formatData(training_imgs, labels)

In [ ]:
# X,y = addRotations(X,y)

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = split(X,y)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from sklearn.ensemble import RandomForestClassifier
from keras import regularizers

In [ ]:
import csv
import matplotlib.pyplot as plt
import time

In [ ]:
batch_size = 64
epochs = 40
num_classes = len(CATEGORIES)

In [ ]:
def plotHistory(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    
def createModel(input_shape=(40, 40, 1)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))

    model.add(MaxPooling2D((2, 2)))
#     model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Dropout(0.4))
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
    model.summary()
    return model

def createModel2(input_shape=(40, 40, 1)):
        # INPUT -> [CONV -> RELU -> CONV -> RELU -> POOL]*3 -> [FC -> RELU]*2 -> FC
    model = Sequential()
    #Input
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     kernel_initializer='he_normal',
                     input_shape=input_shape))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(256, activation='relu'))
    model.add(Dense(256, kernel_regularizer=keras.regularizers.l2(0.3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(256, kernel_regularizer=keras.regularizers.l2(0.2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
    model.summary()
    return model

def leNet4():
    # create model
    model = Sequential()
    
    # Convolution
    model.add(Conv2D(4, (4, 4), input_shape=(40, 40,1), activation='relu'))

    # Subsampling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.1))
    
    # 2nd convolution layer
    model.add( Conv2D(16, (3, 3), activation='relu'))
    
    # Subsamplingmodel.add( Conv2D(64, (2, 2), activation='relu',


    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.1))
    
    model.add(Flatten())
    model.add(Dense(120, kernel_regularizer=keras.regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

#    model.add(Dropout(0.4))
    
    model.add(Dense(128), )
    model.add(BatchNormalization())
    model.add(Activation('relu'))
#     model.add(Dropout(0.4))
    
    model.add(Dense(40), )
    model.add(BatchNormalization())
    model.add(Activation('relu'))

#     model.add(Dropout(0.2))
    
    model.add(Dense(31))
    model.add(Activation('softmax'))
    
    
    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy, 
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])
    
    model.summary()
    return model


In [ ]:
model1 = createModel()
history = model1.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model1.evaluate(X_test, y_test, verbose=0)

In [ ]:
plotHistory(history)

In [ ]:
score

In [ ]:
model1.save_weights('./models/model1/model1_725.h5')

In [ ]:
kaggle_predictions = model1.predict(X_kaggle_2)
ts = time.time()
timestamp = int(ts)
savePredictions('./models/model1/prediction_'+ str(timestamp) + '_' + str(score[1]) + '.csv', kaggle_predictions)

In [ ]:
model2 = createModel2()
history = model2.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
plotHistory(history)
score = model2.evaluate(X_test, y_test, verbose=0)

In [ ]:
score

In [ ]:
preprocessed_kaggle_2 = load(PREPROCESSED_KAGGLE)
X_kaggle_2 = formatXData(list(map(lambda x: x[1], preprocessed_kaggle_2)))
kaggle_predictions_2 = model2.predict(X_kaggle_2)

In [ ]:
ts = time.time()
timestamp = int(ts)
savePredictions('./models/model2/prediction_'+ str(timestamp) + '_' + str(score[1]) + '.csv', kaggle_predictions_2)

In [ ]:
# With L2 regaularization
model2L2Reg = createModel2()
history = model2L2Reg.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
plotHistory(history)
score = model2L2Reg.evaluate(X_test, y_test, verbose=0)

In [ ]:
score

In [ ]:
model2L2Reg.save_weights('./models/model2L2Reg/model2_6775.h5')
model_json = model2L2Reg.to_json()
with open("./models/model2L2Reg/model2_6775.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
kaggle_predictions_2 = model2L2Reg.predict(X_kaggle_2)
ts = time.time()
timestamp = int(ts)
savePredictions('./models/model2L2Reg/prediction_'+ str(timestamp) + '_' + str(score[1]) + '.csv', kaggle_predictions_2)

In [ ]:
outputs = model1.predict(X_val)

In [ ]:
from sklearn.metrics import f1_score

RFClassifier = RandomForestClassifier(n_estimators=94, criterion='gini')
RFClassifier.fit(outputs, list(map(lambda y: np.argmax(y), y_val)))
y_CNN_pred = model1.predict(X_test)
y_RF_pred = RFClassifier.predict(y_CNN_pred)
print(f1_score(list(map(lambda y: np.argmax(y), y_test)), y_RF_pred, average='micro'))
#visualizePredictionsJustWrong(X, y_pred, y, 10, 30, (40, 40))

In [ ]:
lenet = leNet4()
history = lenet.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=200,
          verbose=1,
          validation_data=(X_val, y_val))
plotHistory(history)
score = lenet.evaluate(X_test, y_test, verbose=0)
score

In [ ]:
model2 = createModel2()

In [ ]:
history = model2.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
score

In [ ]:
# preprocessed_kaggle = load(PREPROCESSED_KAGGLE)
# X_kaggle = formatXData(list(map(lambda x: x[1], preprocessed_kaggle)))
# kaggle_predictions = model.predict(X_kaggle)

def savePredictions(outfile, predictions):
    with open(outfile,'w') as csvfile:
        fieldnames = ['Id', 'Category']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for i, prediction in enumerate(predictions):
            index = np.argmax(prediction)
            writer.writerow({'Id': i, 'Category': getCategoryOf(index)})    

In [ ]:
def visualizePredictions(images, predictions, start = 0, end = 10, shape=(40, 40)):
    fig, ax = plt.subplots(figsize=(len(CATEGORIES) + 1, 30))

    for i in list(range(start, end)):

        # plot probabilities:
        ax = plt.subplot2grid((end - start, 5), (i - start, 0), colspan=4);
        plt.bar(np.arange(len(CATEGORIES)), predictions[i], 0.35, align='center');
        plt.xticks(np.arange(len(CATEGORIES)), CATEGORIES)
        plt.tick_params(axis='x', bottom='off', top='off')
        plt.ylabel('Probability')
        plt.ylim(0,1)
        plt.subplots_adjust(hspace = 0.5)

        # plot picture:
        ax = plt.subplot2grid((end - start, 5), (i - start, 4));
        plt.imshow(images[i].reshape(shape),cmap='gray_r', interpolation='nearest');
        plt.xlabel(getCategoryOf(np.argmax(predictions[i]))); # get the label from the dict
        plt.xticks([])
        plt.yticks([])

#     for i, img in enumerate(images):
#         img = img[1].reshape(shape).astype(np.uint8)
#         f, axarr = plt.subplots(nrows=1, ncols=2, figsize=(20, 4))
#         f.suptitle(getCategoryOf(np.argmax(predictions[i])))
#         axarr[0].imshow(img, cmap='gray_r')
#         axarr[0].set_title('Image')
#         axarr[1].bar(range(0, len(CATEGORIES)), predictions[i])
#         plt.show()
def visualizePredictionsJustWrong(images, predictions, actual, start = 0, end = 10, shape=(40, 40)):
    fig, ax = plt.subplots(figsize=(len(CATEGORIES) + 1, 30))
    numFound = 0
    for i in range(start, len(images)):
        if getCategoryOf(np.argmax(predictions[i])) != getCategoryOf(np.argmax(actual[i])):
            # plot probabilities:
            ax = plt.subplot2grid((end - start, 5), (numFound, 0), colspan=4);
            plt.bar(np.arange(len(CATEGORIES)), predictions[i], 0.35, align='center');
            plt.xticks(np.arange(len(CATEGORIES)), CATEGORIES)
            plt.tick_params(axis='x', bottom='off', top='off')
            plt.ylabel('Probability')
            plt.ylim(0,1)
            plt.subplots_adjust(hspace = 0.5)

            # plot picture:
            ax = plt.subplot2grid((end - start, 5), (numFound, 4));
            plt.imshow(images[i].reshape(shape),cmap='gray_r', interpolation='nearest');
            plt.xlabel(getCategoryOf(np.argmax(predictions[i])) + "/" +getCategoryOf(np.argmax(actual[i]))); # get the label from the dict
            plt.xticks([])
            plt.yticks([])
            
            numFound += 1
            if numFound >= end - start:
                return

In [ ]:
ts = time.time()
timestamp = int(ts)
savePredictions('./data/prediction_'+ str(timestamp) + '_' + str(score[1]) + '.csv', kaggle_predictions)

In [ ]:
training_imgs = load(PREPROCESSED_TRAINING)
X,y = formatData(training_imgs, labels)
X_train, y_train, X_test, y_test, X_val, y_val = split(X,y)

In [ ]:
model = createModel()
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1,
          validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
score

In [ ]:
preprocessed_kaggle_1 = load(PREPROCESSED_KAGGLE)
X_kaggle_1 = formatXData(list(map(lambda x: x[1], preprocessed_kaggle_1)))
kaggle_predictions_1 = model.predict(X_kaggle_1)

In [ ]:
visualizePredictions(preprocessed_kaggle_1,kaggle_predictions_1, 10, 30)

In [ ]:
ts = time.time()
timestamp = int(ts)
savePredictions('./data/prediction_'+ str(timestamp) + '_' + str(score[1]) + '.csv', kaggle_predictions)

# Interpolated AND bounded

In [ ]:
PREPROCESSED_TRAINING_BI = "./data/preprocessed_interpolated_bounded.npy"
PREPROCESSED_KAGGLE_BI = "./data/processed_kaggle_interpolated_bounded.npy"

In [ ]:
training_imgs_bi = load(PREPROCESSED_TRAINING_BI)
X,y = formatData(training_imgs_bi, labels)
X_train, y_train, X_test, y_test, X_val, y_val = split(X,y)
model_bi = createModel()
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model_bi.evaluate(X_test, y_test, verbose=0)

In [ ]:
score = model_bi.evaluate(X_test, y_test, verbose=0)
score

In [ ]:
preprocessed_kaggle_bi = load(PREPROCESSED_KAGGLE_BI)
X_kaggle_bi = formatXData(list(map(lambda x: x[1], preprocessed_kaggle_bi)))
kaggle_predictions_bi = model_bi.predict(X_kaggle_bi)

In [ ]:
ts = time.time()
timestamp = int(ts)
savePredictions('./data/prediction_'+ str(timestamp) + '_' + str(score[1]) + '.csv', kaggle_predictions)

In [ ]:
visualizePredictions(preprocessed_kaggle_bi,kaggle_predictions_bi, 10, 30)

In [ ]:
model = createModel2()
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
score

In [ ]:
kaggle_predictions = model.predict(X_kaggle)

In [ ]:
ts = time.time()
timestamp = int(ts)
savePredictions('./data/prediction_'+ str(timestamp) + '_' + str(score[1]) + '.csv', kaggle_predictions)

In [ ]:
visualizePredictions(preprocessed_kaggle[:100],kaggle_predictions[:100])

In [ ]:
# Grabbed from: https://github.com/kradolfer/quickdraw-image-recognition/blob/master/quickdraw_image_recognition.ipynb
def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(42, (5, 5), input_shape=(40, 40, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [ ]:
model_cnn = cnn_model()
# Fit the model
model_cnn.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=200)
# Final evaluation of the model
scores = model_cnn.evaluate(X_test, y_test, verbose=0)

In [ ]:
PREPROCESSED_TRAINING_SMALL = "./data/preprocessed_interpolated_28_28.npy"
PREPROCESSED_KAGGLE_SMALL = "./data/preprocessed_kaggle_interpolated_28_28.npy"

In [ ]:
training_imgs_small = load(PREPROCESSED_TRAINING_SMALL)
X_small,y_small = formatData(training_imgs_small, labels, 28)
X_train, y_train, X_test, y_test, X_val, y_val = split(X_small,y_small)

In [ ]:
model_bi = createModel()
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))
score = model_bi.evaluate(X_test, y_test, verbose=0)

# Bounding Box tests

In [ ]:
PREPROCESSED_TRAINING_BOUNDING_BOX = "./data/train_preprocessed_boundingbox.npy"
PREPROCESSED_KAGGLE_BOUNDING_BOX = "./data/kaggle_preprocessed_boundingbox.npy"
training_imgs_boundingbox = load(PREPROCESSED_TRAINING_BOUNDING_BOX)
X_boundingbox,y_boundingbox = formatData(training_imgs_boundingbox, labels)
X_train_bb, y_train_bb, X_test_bb, y_test_bb, X_val_bb, y_val_bb = split(X_boundingbox,y_boundingbox)
model1_bb = createModel()
history = model1.fit(X_train_bb, y_train_bb,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val_bb, y_val_bb))
score = model1_bb.evaluate(X_test_bb, y_test_bb, verbose=0)